# Assemble the clusterization results

In [1]:
import os
import pickle

sents = []
for filename in os.listdir("sentences"):
    if filename.endswith(".pkl"):
        with open("sentences/"+filename,'rb') as f:
            sents.append(pickle.load(f))
            
            

# Load Conll data

In [2]:
import pandas

pdconll = pandas.read_pickle("Graph_clustering/pdconll_updated.pckl")

In [9]:
pdconll

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,sent_num,row_num,concomp,text,new_id,new_links
index,,,,,,,,,,,,,,,,,,,,
0,1,in,in.01,_,_,_,role=Location|fn=Temporal_collocation|id0=1|ha...,_,3,_,Location,_,_,_,0,0,1,0,1,3
0,2,vn:34_review,review.01,_,_,_,vncls=34|number=SG|id0=5|hasExtArg=yes|vn=vn:3...,_,"1,6",_,"Argument2,Argument1",_,_,_,0,1,1,0,2,"1,6"
0,3,attribute,attribute.01,_,_,_,manner=applied|clause_type=DECL|id0=45|hasExtA...,_,0,_,ROOT,_,_,_,0,2,1,0,3,-1
0,4,vn:11.3_take,take.01,_,_,_,vncls=11.3|fn=Capacity|id0=20|hasExtArg=yes|pl...,_,2,_,Argument2,_,_,_,0,3,1,0,4,2
0,5,misanthrope,misanthrope.01,_,_,_,number=SG|id0=9|hasExtArg=no|vn=misanthrope|pl...,_,2,_,Argument2,_,_,_,0,4,1,0,5,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39278,7,influence,influence.01,_,_,_,fn=Talking_into|id0=8|hasExtArg=yes|member=A1|...,_,5,_,Set,_,_,_,39278,685513,39265,1878,1176,1174
39278,8,bidder,bidder.01,_,_,_,number=SG|id0=16|hasExtArg=yes|vn=bidder|lex=b...,_,"11,6",_,"Argument2,Argument2",_,_,_,39278,685514,39265,1878,1177,"1180,1175"
39278,9,perhaps,perhaps.01,_,_,_,id0=12|hasExtArg=no|vn=perhaps|lex=perhaps_RB_...,_,0,_,ROOT,_,_,_,39278,685515,39265,1878,1178,-39265


# node type list, token vocabulary

In [8]:
ent_types = []
tok_vocab = {}
for _,row in pdconll.iterrows():
    ent_types += row[10].upper().split(",")
    w = row[2][:row[2].find(".")]
    if w not in tok_vocab:
        tok_vocab[w] = len(w)   
    
ent_types = set(ent_types)


# Generate CSV

In [ ]:
#for each text:
#1. Make "Title" from the first sentence
#For the remaining ones:
#2. Enumerate Entities;
#3. Enumerate <Entity Types>
#4. Enumerate E-Rel-E tuples;
#5. Generate the text of the sentence with <entitytype_N>
#6. Generate word_ids of the text with "-1"s instead of the entities

In [124]:
def parse_txt_file(filename):
    with open(filename) as f:
        for line in f:
            yield line.strip().split(" - ",1)
            
parser = parse_txt_file("Graph_clustering/CoNLL2009-ST-train-English_sentences_numbered.txt")            
full_texts = pandas.DataFrame(parser)

In [125]:
full_texts

,0,1
0,0,In an Oct. 19 review of `` The Misanthrope '' ...
1,1,Ms. Haag plays Elianti .
2,2,Rolls - Royce Motor Cars Inc. said it expects ...
3,3,"The luxury auto maker last year sold 1,214 car..."
4,4,"Howard Mosher , president and chief executive ..."
...,...,...
39274,39274,But if the board rejects a reduced bid and dec...
39275,39275,The pilots could play hardball by noting they ...
39276,39276,"If they were to insist on a low bid of , say $..."
39277,39277,"Also , because UAL Chairman Stephen Wolf and o..."


In [151]:


def getOrder(full_texts,sid,txt):
    s_txt = full_texts.iloc[sid][1].lower()
    flag = True
    order = list(range(len(txt)))
    
    while flag:
        flag = False
        for i in range(len(txt)-1):
            t1_pos = s_txt.find(txt[i])
            t2_pos = s_txt.find(txt[i+1])
            
            if t1_pos > -1 and t2_pos > -1:
                if t2_pos < t1_pos:
                    t = txt[i]
                    txt[i] = txt[i+1]
                    txt[i+1] = t
                    
                    t = order[i]
                    order[i] = order[i+1]
                    order[i+1] = t
                    flag = True
    return order        

In [166]:
from tqdm import tqdm
import numpy

def leaveUniq(txt):
    parts = set(txt.split(","))
    return ",".join(list(parts))
    

#buid dict. {tok_id}->{tok_id}->rel_type
#+leave only one cluster per real sentence
def preprocessSents(pd,sents):
    print("Preprocessing..")
    #useful indexes
    sdict_inv = {}
    sdict_dir = {}
    for i,s in enumerate(sents):
        for t_id in s:
            if t_id not in sdict_inv:
                sdict_inv[t_id] = []
            sdict_inv[t_id].append(i)
            if i not in sdict_dir:
                sdict_dir[i] = []
            sdict_dir[i].append(t_id)
                
    #first pass. build real_sent->gen_sents dict
    rsent2sent = {}
    print ("First pass")
    with tqdm(total=len(pd)) as pbar: 
        for _,row in pd.iterrows():
            s_id = row['sent_num']
            if row['new_id'] in sdict_inv:
                cl_ids = sdict_inv[row['new_id']]
                if s_id not in rsent2sent:
                    rsent2sent[s_id] = {}
                for cl_id in cl_ids:    
                    if cl_id not in rsent2sent[s_id]:
                        rsent2sent[s_id][cl_id] = []
                    rsent2sent[s_id][cl_id].append(row['new_id'])
            pbar.update(1)
            
    #remove ambiguation. Just leave the biggest ones
    rsent2sent_filtered = {}
    for s_id in rsent2sent:
        if len(rsent2sent[s_id]) > 1:
            max_id = list(rsent2sent[s_id].keys())[0]
            for cl_id in rsent2sent[s_id]:
                if len(rsent2sent[s_id][cl_id]) > len(rsent2sent[s_id][max_id]):
                    max_id = cl_id
            rsent2sent_filtered[s_id] = rsent2sent[s_id][max_id]        
        else:
            rsent2sent_filtered[s_id] = list(rsent2sent[s_id].values())[0]
    
    #second pass. build dict 
    res_dict = {}
    rel_vocab = {}
    print ("Second pass")
    pd=pd.reset_index().set_index(['new_id','sent_num'])
    with tqdm(total=len(rsent2sent_filtered)) as pbar: 
        for s_id in rsent2sent_filtered:
            ents = rsent2sent_filtered[s_id]
            ents_set = set(ents)
            for e in ents:
                e_row = pd.loc[[e,s_id]]
                #print (e_row['new_links'].iloc[0])
                
                #print (e_row)
                e_links = e_row['new_links'].iloc[0].split(",")

                for l in e_links:
                    if l in ents_set and l > "-1":
                        if e not in res_dict:
                            res_dict[e] = {}
                        l_row = pd.loc[[l,s_id]]
                        link_type = leaveUniq(l_row[10].iloc[0]) + "_" + leaveUniq(e_row[10].iloc[0])
                        if link_type not in rel_vocab:
                            rel_vocab[link_type] = len(rel_vocab)

                        res_dict[e][l] = link_type
            pbar.update(1) 
            #break
    return res_dict, rel_vocab
    

def genIds(txt,tok_vocab):
    return " ".join([str(tok_vocab.get(t,"-1")) for t in txt])
        
def process_conll(pd,rel_vocab,tok_vocab,sents):
    cur_sent_num = None
    cur_txt_num = None

    collect_title = False
    title = []
    t_sent = -1
    txt = []
    p_txt = []
    rels = []
    ents = {}
    ent_strs = []
    ent2num = {}
    ent_types = []
    
    datas = []
    print("Building...")

    with tqdm(total=len(pd)) as pbar: 
        for _,row in pd.iterrows():
            text_num = row["text"]
            sent_num = row["sent_num"]
            if cur_sent_num != None and sent_num != cur_sent_num:
                #Generate line for the output
                if not collect_title:
                    if len(ent2num) > 0:
                        processed = {}
                        for sid in list(ent2num.keys()):
                            sentence = sents[sid]
                            if sid in ent2num:
                                if sid not in processed:
                                    processed[sid] = True
                                    for s in sentence:
                                        if s in ent2num:
                                            processed[s] = True
                                            #print(sid,s,ent2num)
                                            rels.append(str(ent2num[sid]) + " " + str(rel_vocab[sentence[s]]) + " " + str(ent2num[s]))

                    
                    if len(rels) > 0:
                        tmp = []
                        order2 = getOrder(full_texts,t_sent,title)
                        
                        #tmp.append(" ".join(list(numpy.asarray(title)[order2])))
                        tmp.append(" ".join(title))
                        tmp.append(" ; ".join(ent_strs))
                        tmp.append(" ".join(ent_types))
                        tmp.append(" ; ".join(rels))
                        
                        order = getOrder(full_texts,sent_num,p_txt)
                        
                        #tmp.append(" ".join(list(numpy.asarray(txt)[order])))
                        tmp.append(" ".join(txt))
                        tmp.append(genIds(txt,tok_vocab))
                        datas.append(tmp)
        
                    txt = []
                    p_txt = [] 
                    rels = []
                    ents = {}
                    ent_strs = []
                    ent2num = {}
                    ent_types = [] 
                else:
                    collect_title  = False

                cur_sent_num = sent_num
            #else:    
            if text_num != cur_txt_num:
                cur_txt_num = text_num 
                collect_title = True
                cur_sent_num = sent_num
                title = []

            if collect_title:
                #collect first sentence as a title
                t_sent = sent_num
                if row[2].find(".") == -1:
                    title.append(row[1].lower())
                else:    
                    title.append(row[2][:row[2].find(".")].lower())                    
            else:
                tok_id = row['new_id']
                #if it is entity - add it
                if tok_id in sents:
                    #remove duplicates, replace "," to "_"
                    ent_type = "_".join(list(set(row[10].split(","))))
                    ent_str = "<" + ent_type + ">"
                    #if ent_str not in ents:
                    #    ents[ent_str] = []
                    #ents[ent_str] = tok_id   
                    ent2num[tok_id] = len(ent2num)
                    if row[2].find(".") == -1:
                        ent_strs.append(row[1].lower())
                    else:    
                        ent_strs.append(row[2][:row[2].find(".")].lower())
                    ent_types.append(ent_str)
                    txt.append("<" + ent_type + "_"+ str(len(ent2num)-1) +">")
                else:
                    if row[2].find(".") == -1:
                        txt.append(row[1].lower())
                    else:    
                        txt.append(row[2][:row[2].find(".")].lower())
                        
                if row[2].find(".") == -1:
                    p_txt.append(row[1].lower())
                else:    
                    p_txt.append(row[2][:row[2].find(".")].lower())                        

            pbar.update(1)    
            #print (collect_title)
            
    return pandas.DataFrame(datas),datas            
             

In [167]:
dsents, rel_vocab = preprocessSents(pdconll,sents)


  0%|          | 0/685518 [00:00<?, ?it/s]

Preprocessing..
First pass


100%|██████████| 685518/685518 [02:13<00:00, 5133.95it/s]


Second pass


100%|██████████| 34487/34487 [1:24:31<00:00,  6.80it/s]   


In [168]:
gw_dataset, datas = process_conll(pdconll,rel_vocab,tok_vocab,dsents)

  0%|          | 0/685518 [00:00<?, ?it/s]

Building...


100%|██████████| 685518/685518 [02:39<00:00, 4287.01it/s]


In [169]:
gw_dataset

,0,1,2,3,4,5
0,say rolls - royce motor car car inc. in it exp...,mosher ; executive ; chief ; he ; growth ; and...,<Argument1> <Elaboration> <Elaboration> <Argum...,5 18 4 ; 6 19 5 ; 7 19 5 ; 9 21 10,howard <Argument1_0> and say officer president...,-1 -1 3 3 7 9 10 -1 -1 -1 -1 -1 3 -1 -1 -1 3 -...
1,investor to appeal and exchange commission not...,proposal ; ease ; undermine ; for ; and ; comp...,<Argument1> <Argument3> <Argument2> <ROOT> <Ar...,9 18 8 ; 10 19 9 ; 11 19 9,<Argument1_0> <Argument3_1> sec <Argument2_2> ...,-1 -1 -1 -1 11 10 7 9 -1 11 2 -1 9 5 4 7 8 -1 ...
2,investor to appeal and exchange commission not...,in ; about ; change ; from ; executive ; among...,<Location> <NonCore> <Argument2_Argument1> <No...,0 21 1 ; 7 11 3 ; 8 25 4 ; 9 1 5 ; 10 8 7 ; 11...,they make <Location_0> argument letter agency ...,4 4 -1 8 6 6 -1 -1 4 6 7 -1 -1 -1 -1 -1 -1 4 -...
3,investor to appeal and exchange commission not...,change ; allow ; report ; executive ; exercise...,<Argument2_Argument1> <Argument1> <Argument2> ...,2 26 1 ; 3 17 2 ; 4 8 2 ; 5 9 2 ; 6 8 4 ; 7 28...,<Argument2_Argument1_0> <Argument1_1> propose ...,-1 -1 7 -1 4 -1 -1 -1 -1 -1 -1 4
4,investor to appeal and exchange commission not...,many ; and ; shake ; prompt ; stack ; individu...,<Argument1> <Argument2> <Set_Argument1> <Elabo...,2 54 1 ; 3 16 1 ; 4 54 1,<Argument1_0> maintain of <Argument2_1> letter...,-1 8 2 -1 7 -1 -1 -1 5 10 2 8 -1 3 6 7 7 2 6 4...
...,...,...,...,...,...,...
22325,donald trump withdraw face doubt his before $ ...,8 ; million ; in ; financing,<ROOT> <ROOT> <ROOT> <Argument1>,1 15 0 ; 2 15 0 ; 3 81 1,together receive and $ raise chase citicorp <R...,8 7 3 1 5 5 -1 -1 -1 -1 4 3 -1
22326,donald trump withdraw face doubt his before $ ...,break ; coalition ; now ; of ; and ; longtime ...,<ROOT> <Argument1> <Time> <ROOT> <Argument2> <...,1 8 0 ; 2 54 1 ; 3 39 1 ; 4 61 3 ; 7 50 6,in addition <ROOT_0> <Argument1_1> apart <Time...,2 8 -1 -1 5 -1 -1 -1 10 5 5 -1 4 -1 5 -1 4 2 3...
22327,donald trump withdraw face doubt his before $ ...,outside ; in ; top ; chicago ; meeting,<Location> <Location> <ROOT> <Argument2> <Argu...,1 49 0 ; 4 15 2,group resilience 's get today test convene fir...,5 10 2 3 5 4 7 5 3 6 -1 -1 5 2 -1 -1 -1 5 8 10
22328,donald trump withdraw face doubt his before $ ...,member ; overwhelmingly ; why ; explain ; last...,<Argument1> <Manner> <Purpose> <Argument2> <RO...,1 7 2 ; 3 39 2 ; 4 54 2 ; 5 100 4 ; 6 49 3 ; 7...,buy out finance approve banks refuse week <Arg...,3 3 7 7 5 6 4 -1 -1 -1 -1 -1 -1 -1 4 -1 -1 5 -...


In [113]:
from sklearn.model_selection import train_test_split

gw_dataset_train, gw_dataset_test = train_test_split(gw_dataset, test_size=0.2, random_state=1)

#gw_dataset_train, gw_dataset_val = train_test_split(gw_dataset_train, test_size=0.25, random_state=1)

In [171]:
gw_dataset.to_csv("conll_train.tsv",header=False,index=False,sep="\t")
#gw_dataset_test.to_csv("conll_test.tsv",header=False,index=False,sep="\t")
#gw_dataset_val.to_csv("conll_val.tsv",header=False,index=False,sep="\t")

with open("relations.vocab","w") as f:
    for r in rel_vocab:
        f.write(r+"\n")


In [121]:
gw_dataset_train[4].to_csv("conll_gold.tsv",header=False,index=False,sep="\t")

In [87]:
et = []
et2 = []
for e in ent_types:
    et.append("<"+e+">")
    et2.append(e)

In [88]:
et

['<ROOT>',
 '<DIRECTION>',
 '<BENEFACTIVE>',
 '<ARGUMENT1>',
 '<ARGUMENT2>',
 '<ARGUMENT6>',
 '<ARGUMENT10>',
 '<MANNER>',
 '<EXTENT>',
 '<ELABORATION>',
 '<SET>',
 '<ARGUMENT3>',
 '<LOCATION>',
 '<ARGUMENT4>',
 '<NAME>',
 '<PURPOSE>',
 '<TIME>',
 '<NONCORE>']

In [89]:
et2

['ROOT',
 'DIRECTION',
 'BENEFACTIVE',
 'ARGUMENT1',
 'ARGUMENT2',
 'ARGUMENT6',
 'ARGUMENT10',
 'MANNER',
 'EXTENT',
 'ELABORATION',
 'SET',
 'ARGUMENT3',
 'LOCATION',
 'ARGUMENT4',
 'NAME',
 'PURPOSE',
 'TIME',
 'NONCORE']

In [170]:
len(rel_vocab)

104